# Pretraživanje informacija

Retke matrice se pojavljuju u mnogim domenima. Sistemi preporuka, obrada teksta, kompjuterska vizija ili grafovska modelovanja su samo neki od njih. U nastavku će biti prikazan jedan pojednostavljen sistem za pretraživanje informacija koji za zadati upit korisnika treba da pronađe najrelevantniji dokument kolekcije dokumenata. <img src='assets/ir_system.png' style='height: 300px;'>

Za razliku od reprezentacije teksta koju smo koristiti u zadatku sumarizacije, u ovom problemu ćemo za reprezentaciju dokumenata i upita koristiti Tf-Idf vektorizacije. 

### TF-IDF reprezentacija

Prvi korak u obradi kolekcije dokumenata je kreiranje `vokabulara`. Ovaj zadatak podrazumeva izdvajanje svih reči koje se nalaze u kolekciji dokumenata i njihovo leksikografsko sortiranje. U realnim zadacima, u zavisnosti od kolekcije sa kojom se radi, veličina vokabulara može varirati od 10000 do 100000 reči. Da bi se redukovala dimenzija vokabulara, samo izdvajenje reči može da prati filtriranje dopustivih reči, na primer, izbacivanje brojeva, interpunkcije, reči koje se retko pojavljuju poput pravopisnih propusta, reči koje se često pojavljuju poput članova ili predloga. Takođe, izdvajanje reči može pratiti i njihova normalizacija. Na primer, reči *playing*, *played* i *plays* se mogu svesti na istu lingvističku formu (lemu) *play* ili na neki drugi veštački koren (engl. stem), na primer *pla*, koji bi ukazao na zajedničko značenje. 

Svaki dokument polazne kolekcije je dalje potrebno izraziti vektorski u terminima izdvojenog vokabulara tako što se na pozicijima koje odgovaraju rečima koje sadrže upisuju odgovorajuće težine. Na ostalim pozicijama se upisuju nule pa su dobijene reprezentacije dokumenata vrlo retke. 

<img src='assets/tfidf_matrix.png'>

Težine koje odgovaraju pojedinačnim rečima se mogu računati na razne načine. TF-IDF reprezentacija očekuje da se u obzir uzme frekvencija same reči (engl. term frequency) u dokumentu, kao i njena inverzna frekvencija dokumenta (engl. inverse document frequency) kojom se ukazuje na prisutnost reči u celoj kolekciji dokumenata. 

Težina *tf-idf* se računa kao proizvod faktora $Tf(t, d)$ i $Idf(t, D)$.

$Tf(t, d)$ predstavlja količnik broja pojavljivanja termina (reči) $t$ u dokumentu $d$ i ukupnog broja reči u dokumentu $d$ i računa se kao
$$Tf(t, d) = \frac{count(t)}{|d|}$$

$Idf(t, D)$ predstavlja logaritam količnika ukupnog broja dokumenata skupa $D$ i broja dokumenata (iz skupa 
$D$) u kojima se pojavljuje termin (reč) $t$ i računa se kao
$$Idf(t, D) = log(\frac{|D|}{N_t})$$


$$TfIdf(t, d, D) =Tf(t, d)\cdot Idf(t, D)$$

Važnost termina (reči) raste sa rastom broja njegovog pojavljivanja u dokumentu, ali se otežava njegovim ukupnim brojem pojavljivanja u celokupnom skupu dokumenata (rečima koje se pojavljuju u većem broju dokumenata pridružuje se manja vrednost jer je i njihova diskriminativnost manja).

Postoje i mnoge varijacije na temu vrednosti ovih faktora o kojima se može više pročitati u članku na [Vikipediji](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) ili u [prezentaciji](https://slideplayer.com/slide/11565250/) koja prati sadržaj knjige *Natural Language and Speech Processing* autora Dana Džurafskog. 

In [1]:
import numpy as np
from scipy import sparse

Mi ćemo za računanje ovakvih reprezentacija iskoristiti podršku `scikit-learn` biblioteke i njenu klasu `TfIdVectorizer`. 

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

Kolekcija dokumenata sa kojom ćemo raditi je poznata pod imenom `20newsgroup`. Ona sadrži kolekciju 18000 *newsgroup* postova razvrstanih u 20 kategorija. Mi ćemo u radu upotrebiti dve kategorije, `sci.space` i `comp.graphics`.

In [3]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.graphics', 'sci.space']
documents = fetch_20newsgroups(subset='train', categories=categories)

Možemo pogledati kako izgleda jedan dokument kolekcije.

In [4]:
print(documents.data[100])

From: newmme@helios.tn.cornell.edu (Mark E. J. Newman)
Subject: HELP: advice on what video system to buy
Keywords: video, RS6000
Organization: Cornell University
Lines: 22

If this question is covered elsewhere, I apologize, but I need information
fast.

My department has been given a large sum of money to install a video system
on our network of IBM RS6000 workstations.  This is not an area in which I
have any expertise, so I wonder if anyone out there can offer advice.  We
would like a system, based either on VHS or 8mm video which will allow one 
write video, frame by frame on tape for play-back in real time.  It's for
visualization of physics problems.  Can anyone tell me what hardware is
available which would work for our system?  Some support software is
obviously needed too, but nothing particularly sophisticated, since the
software we actually use for the visualization is all already written.

Please email with replies, as I don't read this group.  Many thanks for your
help.

D

Tj.

In [5]:
documents.data[100]

"From: newmme@helios.tn.cornell.edu (Mark E. J. Newman)\nSubject: HELP: advice on what video system to buy\nKeywords: video, RS6000\nOrganization: Cornell University\nLines: 22\n\nIf this question is covered elsewhere, I apologize, but I need information\nfast.\n\nMy department has been given a large sum of money to install a video system\non our network of IBM RS6000 workstations.  This is not an area in which I\nhave any expertise, so I wonder if anyone out there can offer advice.  We\nwould like a system, based either on VHS or 8mm video which will allow one \nwrite video, frame by frame on tape for play-back in real time.  It's for\nvisualization of physics problems.  Can anyone tell me what hardware is\navailable which would work for our system?  Some support software is\nobviously needed too, but nothing particularly sophisticated, since the\nsoftware we actually use for the visualization is all already written.\n\nPlease email with replies, as I don't read this group.  Many than

Kolekciju ćemo pripremiti tako što ćemo eliminisati početne *From:*, *Subject:*, *Keywords:", *Organization:*, *Lines:* i *Distribution:* linije i zameniti pojavljivanja znaka za novi red belinom. 

In [6]:
def document_cleaner(document):
    clean_document = []
    
    lines = document.split('\n')
    for line in lines:
        if  line.startswith('From:') or \
            line.startswith('Subject:') or \
            line.startswith('Keywords:') or \
            line.startswith('Organization:') or \
            line.startswith('Lines:') or \
            line.startswith('Distribution:'):
            continue
        else:
            clean_line = line.replace('\n>', ' ')
            clean_line = clean_line.replace('\n', ' ')
            clean_document.append(clean_line)
            
    return " ".join(clean_document)

In [7]:
document_cleaner(documents.data[100])

" If this question is covered elsewhere, I apologize, but I need information fast.  My department has been given a large sum of money to install a video system on our network of IBM RS6000 workstations.  This is not an area in which I have any expertise, so I wonder if anyone out there can offer advice.  We would like a system, based either on VHS or 8mm video which will allow one  write video, frame by frame on tape for play-back in real time.  It's for visualization of physics problems.  Can anyone tell me what hardware is available which would work for our system?  Some support software is obviously needed too, but nothing particularly sophisticated, since the software we actually use for the visualization is all already written.  Please email with replies, as I don't read this group.  Many thanks for your help.  Dr. M. E. J. Newman. Department of Physics, Cornell University. newmme@helios.tn.cornell.edu   "

In [8]:
clean_documents = [document_cleaner(document) for document in documents.data]

Ukupna broj dokumenata u kolekciji je: 

In [9]:
len(clean_documents)

1177

`TfIdfVectorizer` će za zadatu kolekciju kreirati Tf-Idf matricu. Opciono mu se mogu proslediti parametri podešavanja, na primer, mi ćemo isključiti sve reči koje imaju manje od 5 pojavljivanja (parametar `min_df` sa vrednošću 5), sve reči ćemo svesti na zapis malim slovima (parametar `lowercase` sa podrazumevanom vrednošću True), isključićemo stop reči za engleski jezik (parametar `stop_words` sa vrednošću english), a pod rečima ćemo smatrati niske karaktera opisane regularnim izrazom \\b\\w\\w+\\b (podrazumevana vrednost `token_pattern` parametra).

In [10]:
vectorizer = TfidfVectorizer(min_df=5, stop_words='english')

In [11]:
vectorizer.fit(clean_documents)

TfidfVectorizer(min_df=5, stop_words='english')

Izdvojene reči vokabulara se mogu pročitati preko svojstva `vocabulary_` koje predstavlja rečnik svi reči i njihovog broja pojavljivanja. 

In [12]:
len(vectorizer.vocabulary_)

4746

Leksikografski uređen spisak reči se može dobiti metodom `get_feature_names`. Ispod su prikazane reči na pozicijama od 1000 do 1020.

In [13]:
vectorizer.get_feature_names()[1000:1020]

['clouds',
 'club',
 'clue',
 'cm',
 'cmu',
 'coast',
 'code',
 'codec',
 'coded',
 'codes',
 'coffee',
 'coffman',
 'col',
 'cold',
 'collapse',
 'collect',
 'collecting',
 'collection',
 'collections',
 'college']

Analizom ovih reči se mogu dobiti još neke ideje koje se tiču čišćenja i pripreme teksta. Na primer, možete probati da izbacite brojeve iz vokabulara modifikacijom `token_pattern` regularnog izraza. 

Mi ćemo korišćenjem ovog vektorizatora dalje transformisati celu kolekciju u Tf-Idf matricu.

In [14]:
vectorized_documents = vectorizer.transform(clean_documents)

Možemo ispitati strukturu ovako dobijene matrice.

Matrica koja se generiše je u CSR formatu.

In [15]:
type(vectorized_documents)

scipy.sparse.csr.csr_matrix

Svaki dokument je opisan vektorom dužine 4746.

In [16]:
vectorized_documents.shape

(1177, 4746)

Možemo izračunati i prosečan broj ne-nula vrednosti po dokumentu. 

In [17]:
nnz_per_document = vectorized_documents.nnz / vectorized_documents.shape[0]
nnz_per_document

74.04078164825829

Sve navedene informacije nam govore da je dobijena matrica izrazito retka, sa oko 1.6% ne-nula vrednosti po dokumentu. 

Recimo da nas interesuju dokumenti koji odgovaraju upitu `computer science`. Upit ćemo pripremiti na isti način kao što smo pripremili i dokumente kako bi dobili iste reprezentacije.

In [18]:
query = "computer science"

In [19]:
vectorized_query = vectorizer.transform([query])

Ostaje još da pronađemo dokument koji je najrelevantniji za zadati upit. Iskoristićemo kosinusnu sličnost kao meru preklapanja upita `query` i dokumenta `document`. Kosinusna sličnost predstavlja vrednost kosinusa ugla između vektorskih reprezentacija dokumenata `document` i upita `query`. <img src='assets/cosine_similarity.png'>

In [20]:
def cosine_similarity(document, query):
    similarity = np.inner(document, query.reshape(-1,1)) / (sparse.linalg.norm(document) * sparse.linalg.norm(query))
    
    # rezultat skalarnog prozivoda je retki vektor dizmenzije 1x1 pa je ceo rezultat retki vektor dimenzije 1x1
    # zbog toga pozivamo funkciju toarray() i ocitavamo cisto numericku vrednost slicnosti
    return similarity.toarray()[0][0]

Dalje računamo vrednosti sličnosti za sve dokumente i zadati upit.

In [21]:
scores = []
for i in range (0, vectorized_documents.shape[0]):
    score = cosine_similarity(vectorized_documents.getrow(i), vectorized_query)
    scores.append(score)

/opt/anaconda3/lib/python3.7/site-packages/scipy/sparse/base.py:581: RuntimeWarning: divide by zero encountered in double_scalars
  return self.astype(np.float_)._mul_scalar(1./other)


Rangiranje dokumenata vršimo po relevantnosti tj. po vrednosti kosinusne sličnosti, od najrelevantnijeg ka najmanje relevantnom dokumentu.

In [22]:
ranking = np.argsort(scores)[::-1]

Naš sistem bi vratio sledeći rezultat iz originalne kolekcije:

In [23]:
documents.data[ranking[0]]

"From: greg@cs.uct.ac.za (Gregory Torrance)\nSubject: Automatic layout of state diagrams\nOrganization: Computer Science Department, University of Cape Town\nLines: 18\n\nHi,\n\nI'm hoping someone out there will be able to help our computer science\nproject group. We are doing computer science honours, and our project\nis to do a 'graphical simulator for a finite state automata'.\n\nBasically, the program must draw a diagram of a FSA from a textual grammar,\nshowing circles for states, and labeled arc's in-between.\n\nThe problem is working out the best way to layout the states, and draw the\narc's in-between so that as few arc's as possible cross each other.\n\nIf anyone has any suggestions/algorithms/bug-free ready to compile C code :) \nthat might help us, it would be much appreciated.\n\nThanks in advance,\n\nGregory\n"

Izračunata sličnost ima vrednost:

In [24]:
scores[ranking[0]]

0.276958928131604

Redni broj najrelevantnijeg dokumenta u kolekciji je:

In [25]:
ranking[0]

545

Alternativno, za računanje kosinusne sličnosti mogli smo iskoristiti i ugrađenju funkciju `pairwise.cosine_similarity` paketa `metrics`.

In [26]:
from sklearn.metrics import pairwise

In [27]:
new_scores = pairwise.cosine_similarity(vectorized_documents, vectorized_query)

Najveća ocena je dobijena za dokument:

In [28]:
np.argmax(new_scores)

545

Vrednost ocene je: 

In [29]:
np.max(new_scores)

0.276958928131604